# 선형 회귀(Linear Regression)
1. 선형 회귀
2. 자동 미분
3. 다중 선형 회귀
4. nn.Module로 구현하는 선형 회귀
5. 클래스로 파이토치 모델 구현하기
6. 미니 배치와 데이터 로드
7. 커스텀 데이터셋

---

## 선형회귀

### 1. 선형 회귀의 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 현재 실습하고 있는 파이썬 코드를 재실행해도 다음에도 같은 결과가 나오도록 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(1)

##### 1. 변수선언

In [3]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

##### 2. 가중치와 편향의 초기화

In [4]:
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True)
# 가중치 W를 출력
print(W) 
print(b)

tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)


##### 3. 가설 설정

In [5]:
hypothesis = W * x_train + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


##### 4. 비용 함수 선언

In [6]:
cost = torch.mean((hypothesis - y_train) ** 2) 
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


##### 5. 경사 하강법 구현

In [7]:
optimizer = optim.SGD([W, b], lr = 0.01)

In [8]:
""" 항상 따라다니는 코드(epoch마다)
# gradient를 0으로 초기화
optimizer.zero_grad() 
# 비용 함수를 미분하여 gradient 계산
cost.backward() 
# W와 b를 업데이트
optimizer.step() 
"""

' 항상 따라다니는 코드(epoch마다)\n# gradient를 0으로 초기화\noptimizer.zero_grad() \n# 비용 함수를 미분하여 gradient 계산\ncost.backward() \n# W와 b를 업데이트\noptimizer.step() \n'

In [9]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 2000 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/2000 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/2000 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/2000 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/2000 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/2000 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/2000 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/2000 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/2000 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/2000 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/2000 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/2000 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/2000 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/2000 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/2000 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/2000 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/2000 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/2000 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/2000 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/2000 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/2000 W: 1.997, b: 0.008 Cost: 0.000008

---

### 2.자동 미분 실습하기

##### 1. 자동 미분(Autograd) 실습하기

In [10]:
# 텐서에 대한 기울기를 저장하겠다는 의미
w = torch.tensor(2.0, requires_grad=True)

y = w**2
z = 2*y + 5

# 해당 수식을 w에 대하여 미분
z.backward()

print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 8.0


---

### 3.다중 선형 회귀

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [12]:
# 훈련 데이터
x1_train = torch.FloatTensor([[77], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[88], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [13]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [14]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.296 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.676 w2: 0.682 w3: 0.682 b: 0.008 Cost: 13.648926
Epoch  200/1000 w1: 0.690 w2: 0.698 w3: 0.698 b: 0.007 Cost: 12.305525
Epoch  300/1000 w1: 0.702 w2: 0.714 w3: 0.714 b: 0.007 Cost: 11.097908
Epoch  400/1000 w1: 0.714 w2: 0.729 w3: 0.729 b: 0.007 Cost: 10.012353
Epoch  500/1000 w1: 0.725 w2: 0.744 w3: 0.744 b: 0.007 Cost: 9.036519
Epoch  600/1000 w1: 0.736 w2: 0.757 w3: 0.757 b: 0.007 Cost: 8.159314
Epoch  700/1000 w1: 0.746 w2: 0.770 w3: 0.770 b: 0.007 Cost: 7.370742
Epoch  800/1000 w1: 0.755 w2: 0.782 w3: 0.782 b: 0.007 Cost: 6.661847
Epoch  900/1000 w1: 0.764 w2: 0.794 w3: 0.794 b: 0.007 Cost: 6.024539
Epoch 1000/1000 w1: 0.773 w2: 0.805 w3: 0.805 b: 0.007 Cost: 5.451605


### 행렬 연산을 고려하여 파이토치로 구현하기

In [15]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros((5, 1), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2550, 80.8370, 79.6496, 86.7367, 61.6576]) Cost: 9299.041992
Epoch    2/20 hypothesis: tensor([104.9094, 126.0961, 124.2437, 135.2988,  96.1786]) Cost: 2916.040283
Epoch    3/20 hypothesis: tensor([125.9912, 151.4358, 149.2109, 162.4876, 115.5064]) Cost: 915.195618
Epoch    4/20 hypothesis: tensor([137.7942, 165.6231, 163.1894, 177.7099, 126.3278]) Cost: 288.001404
Epoch    5/20 hypothesis: tensor([144.4023, 173.5665, 171.0157, 186.2326, 132.3867]) Cost: 91.397415
Epoch    6/20 hypothesis: tensor([148.1018, 178.0139, 175.3974, 191.0042, 135.7791]) Cost: 29.768749
Epoch    7/20 hypothesis: tensor([150.1729, 180.5040, 177.8505, 193.6756, 137.6786]) Cost: 10.449845
Epoch    8/20 hypothesis: tensor([151.3323, 181.8983, 179.2240, 195.1713, 138.7423]) Cost: 4.393543
Epoch    9/20 hypothesis: tensor([151.9812, 182.6791, 179.9928, 196.0086, 139.3380]) Cost: 2.494627
Epoch   10/20 hypo

---

### 4.nn.Module로 구현하는 선형 회귀

##### 1. 단순 선형 회귀 구현하기

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [17]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [18]:
model = nn.Linear(1,1)
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [20]:
nb_epochs = 2000
for epoch in range(1, nb_epochs+1):
    prediction = model(x_train)
    
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()))
        #print(list(model.parameters()))

Epoch  100/2000 Cost: 1.237737
Epoch  200/2000 Cost: 6.770324
Epoch  300/2000 Cost: 12.201328
Epoch  400/2000 Cost: 0.356613
Epoch  500/2000 Cost: 8.695301
Epoch  600/2000 Cost: 10.845249
Epoch  700/2000 Cost: 0.008479
Epoch  800/2000 Cost: 10.454887
Epoch  900/2000 Cost: 9.149695
Epoch 1000/2000 Cost: 0.217620
Epoch 1100/2000 Cost: 11.909933
Epoch 1200/2000 Cost: 7.246561
Epoch 1300/2000 Cost: 0.976930
Epoch 1400/2000 Cost: 12.929339
Epoch 1500/2000 Cost: 5.302049
Epoch 1600/2000 Cost: 2.219204
Epoch 1700/2000 Cost: 13.427772
Epoch 1800/2000 Cost: 3.482737
Epoch 1900/2000 Cost: 3.833276
Epoch 2000/2000 Cost: 13.373055


In [21]:
print(list(model.parameters()))
print(model(torch.FloatTensor([[20]])))

[Parameter containing:
tensor([[3.4427]], requires_grad=True), Parameter containing:
tensor([0.4733], requires_grad=True)]
tensor([[69.3268]], grad_fn=<AddmmBackward>)


##### 2. 다중 선형 회귀 구현하기

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [23]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [24]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
nb_epochs = 2000

for epoch in range(1, nb_epochs):
    cost = torch.nn.functional.mse_loss(y_train, model(x_train))
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch  100/2000 Cost: 0.226011
Epoch  200/2000 Cost: 0.223930
Epoch  300/2000 Cost: 0.221953
Epoch  400/2000 Cost: 0.220074
Epoch  500/2000 Cost: 0.218288
Epoch  600/2000 Cost: 0.216593
Epoch  700/2000 Cost: 0.214968
Epoch  800/2000 Cost: 0.213431
Epoch  900/2000 Cost: 0.211972
Epoch 1000/2000 Cost: 0.210572
Epoch 1100/2000 Cost: 0.209247
Epoch 1200/2000 Cost: 0.207987
Epoch 1300/2000 Cost: 0.206780
Epoch 1400/2000 Cost: 0.205629
Epoch 1500/2000 Cost: 0.204536
Epoch 1600/2000 Cost: 0.203490
Epoch 1700/2000 Cost: 0.202495
Epoch 1800/2000 Cost: 0.201550
Epoch 1900/2000 Cost: 0.200642


In [25]:
print(list(model.parameters()))

new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

[Parameter containing:
tensor([[0.9778, 0.4538, 0.5768]], requires_grad=True), Parameter containing:
tensor([0.2802], requires_grad=True)]
훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.2304]], grad_fn=<AddmmBackward>)


---

### 05. 클래스로 파이토치 모델 구현하기

##### 1. 모델을 클래스로 구현하기

In [26]:
class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
    def __init__(self): #
        super().__init__()
        self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim=1, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [27]:
model = LinearRegressionModel()

In [28]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    def forward(self, x):
        return self.linear(x)

In [29]:
model = MultivariateLinearRegressionModel()

##### 2. 단순 선형 회귀 클래스로 구현하기

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

nb_epochs = 2000
for epoch in range(nb_epochs+1):
    h = model.forward(x_train)
    cost = nn.functional.mse_loss(h, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 13.103540
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [32]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [33]:
class MultiVariableLinearRegression(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)
    
    def forward(self, x):
        return self.linear(x)
        

In [34]:
model = MultiVariableLinearRegression(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5 )
nb_epochs = 2000

for epoch in range(nb_epochs+1):
    predict = model.forward(x_train)
    cost = torch.mean(nn.functional.mse_loss(predict, y_train))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("{:4d} / {}: \tcost: {:.6f}".format(epoch, nb_epochs, cost.item()))

   0 / 2000: 	cost: 31667.597656
 100 / 2000: 	cost: 0.225993
 200 / 2000: 	cost: 0.223911
 300 / 2000: 	cost: 0.221941
 400 / 2000: 	cost: 0.220059
 500 / 2000: 	cost: 0.218271
 600 / 2000: 	cost: 0.216575
 700 / 2000: 	cost: 0.214950
 800 / 2000: 	cost: 0.213413
 900 / 2000: 	cost: 0.211952
1000 / 2000: 	cost: 0.210559
1100 / 2000: 	cost: 0.209230
1200 / 2000: 	cost: 0.207967
1300 / 2000: 	cost: 0.206762
1400 / 2000: 	cost: 0.205618
1500 / 2000: 	cost: 0.204529
1600 / 2000: 	cost: 0.203481
1700 / 2000: 	cost: 0.202486
1800 / 2000: 	cost: 0.201539
1900 / 2000: 	cost: 0.200634
2000 / 2000: 	cost: 0.199770


----

### 6.미니배치와 데이터 로드

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [36]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [37]:
dataset = TensorDataset(x_train, y_train)

In [38]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [39]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx)
        # print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 46978.785156
Epoch    0/20 Batch 2/3 Cost: 6497.784668
Epoch    0/20 Batch 3/3 Cost: 5174.395996
Epoch    1/20 Batch 1/3 Cost: 711.106018
Epoch    1/20 Batch 2/3 Cost: 401.381378
Epoch    1/20 Batch 3/3 Cost: 212.294891
Epoch    2/20 Batch 1/3 Cost: 26.882061
Epoch    2/20 Batch 2/3 Cost: 0.057693
Epoch    2/20 Batch 3/3 Cost: 41.471272
Epoch    3/20 Batch 1/3 Cost: 13.444824
Epoch    3/20 Batch 2/3 Cost: 14.656877
Epoch    3/20 Batch 3/3 Cost: 5.495739
Epoch    4/20 Batch 1/3 Cost: 6.619390
Epoch    4/20 Batch 2/3 Cost: 19.459251
Epoch    4/20 Batch 3/3 Cost: 16.812302
Epoch    5/20 Batch 1/3 Cost: 19.544233
Epoch    5/20 Batch 2/3 Cost: 21.681820
Epoch    5/20 Batch 3/3 Cost: 13.138634
Epoch    6/20 Batch 1/3 Cost: 2.338879
Epoch    6/20 Batch 2/3 Cost: 21.505651
Epoch    6/20 Batch 3/3 Cost: 17.908745
Epoch    7/20 Batch 1/3 Cost: 8.261931
Epoch    7/20 Batch 2/3 Cost: 12.769864
Epoch    7/20 Batch 3/3 Cost: 24.187618
Epoch    8/20 Batch 1/3 Cost: 10.52

In [40]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[156.7833]], grad_fn=<AddmmBackward>)


---

### 7.커스텀 데이터셋

In [41]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self):
    #데이터셋의 전처리를 해주는 부분
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    def __len__(self):
    #데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        return len(self.x_data)

    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y        
    #데이터셋에서 특정 1개의 샘플을 가져오는 함수
    
    #len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
    #dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item

In [42]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [43]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [44]:
nb_epoch = 2000
for epoch in range(nb_epoch+1):
    for batch_idx, samples in enumerate(dataloader):
        x, y = samples
        h = model(x)
        cost = torch.nn.functional.mse_loss(h, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        if epoch % 100 ==0:
            print("epoch: {:04d}/{:4d}\tbatch: {:03d}/{:03d}\tcost{:.6f}".format(
                epoch, nb_epoch, batch_idx + 1, len(dataloader), cost.item())
            )
        
        

epoch: 0000/2000	batch: 001/003	cost6319.331543
epoch: 0000/2000	batch: 002/003	cost1066.903931
epoch: 0000/2000	batch: 003/003	cost796.235840
epoch: 0100/2000	batch: 001/003	cost1.625426
epoch: 0100/2000	batch: 002/003	cost1.603504
epoch: 0100/2000	batch: 003/003	cost3.733830
epoch: 0200/2000	batch: 001/003	cost2.425547
epoch: 0200/2000	batch: 002/003	cost1.391298
epoch: 0200/2000	batch: 003/003	cost0.552497
epoch: 0300/2000	batch: 001/003	cost2.272011
epoch: 0300/2000	batch: 002/003	cost1.876806
epoch: 0300/2000	batch: 003/003	cost0.687435
epoch: 0400/2000	batch: 001/003	cost1.741325
epoch: 0400/2000	batch: 002/003	cost0.630343
epoch: 0400/2000	batch: 003/003	cost1.476142
epoch: 0500/2000	batch: 001/003	cost1.645198
epoch: 0500/2000	batch: 002/003	cost0.941095
epoch: 0500/2000	batch: 003/003	cost0.352050
epoch: 0600/2000	batch: 001/003	cost0.980761
epoch: 0600/2000	batch: 002/003	cost1.313380
epoch: 0600/2000	batch: 003/003	cost0.637371
epoch: 0700/2000	batch: 001/003	cost1.142746
ep